In [1]:
import pandas as pd

demographics = pd.read_csv("us_census/demographics.csv")
demographics.head()

,zip,lat,lng,city,state_id,state_name,population,density,county_name,po_box,dist_highway,dist2_large_airport,dist2_medium_airport,dist_to_shore,number_of_business,adjusted_gross_income,total_income_amount,number_of_returns
0,1001,42.06262,-72.62521,Agawam,MA,Massachusetts,16088,550.1,Hampden,0,1.387035,106.145765,12.946212,93.049251,438.0,598807,6.047690e+05,9320
1,1002,42.37633,-72.46462,Amherst,MA,Massachusetts,27323,198.1,Hampshire,0,14.438177,112.264368,21.080079,133.370144,571.0,989558,1.005796e+06,9880
2,1005,42.42117,-72.10655,Barre,MA,Massachusetts,4947,44.2,Worcester,0,16.788339,90.664964,25.547718,97.639881,97.0,164207,1.660540e+05,2490
3,1007,42.28163,-72.40009,Belchertown,MA,Massachusetts,15304,107.7,Hampshire,0,13.663839,101.552921,14.762395,114.406034,217.0,647074,6.547390e+05,7970
4,1008,42.18234,-72.95819,Blandford,MA,Massachusetts,1171,7.4,Hampden,0,2.593655,136.548797,20.177950,107.466779,18.0,47826,4.824100e+04,660


In [2]:
# demographics = demographics[demographics['state_id'] == "PA"]
demographics = demographics[['zip', 'dist_highway', 'adjusted_gross_income']]
demographics = demographics.rename(columns={"zip":"postal_code"})
demographics.head()

,postal_code,dist_highway,adjusted_gross_income
0,1001,1.387035,598807
1,1002,14.438177,989558
2,1005,16.788339,164207
3,1007,13.663839,647074
4,1008,2.593655,47826


In [3]:
unemployment = pd.read_csv("us_census/unemployment.csv")
unemployment.head()

,FIPS_Code,State,Area_Name,Attribute,Value
0,0,US,United States,Civilian_labor_force_2000,142601576.0
1,0,US,United States,Employed_2000,136904853.0
2,0,US,United States,Unemployed_2000,5696723.0
3,0,US,United States,Unemployment_rate_2000,4.0
4,0,US,United States,Civilian_labor_force_2001,143786537.0


In [4]:
# unemployment = unemployment[unemployment['State'] == "PA"]
# Get only 2023 data
unemployment = unemployment[unemployment['Attribute'].str.contains(
    '2023.*') == True].reset_index()
unemployment = unemployment.pivot(
    index=['FIPS_Code', 'State'], columns='Attribute', values='Value')
unemployment = unemployment.reset_index()
unemployment.columns.name = None
unemployment.head()

,FIPS_Code,State,Civilian_labor_force_2023,Employed_2023,Metro_2023,Rural_Urban_Continuum_Code_2023,Unemployed_2023,Unemployment_rate_2023
0,0,US,166689556.0,160623678.0,NaN,NaN,6065878.0,3.6
1,1000,AL,2308131.0,2250472.0,NaN,NaN,57659.0,2.5
2,1001,AL,27000.0,26409.0,1.0,2.0,591.0,2.2
3,1003,AL,104409.0,102029.0,1.0,3.0,2380.0,2.3
4,1005,AL,7944.0,7592.0,0.0,6.0,352.0,4.4


In [5]:
zipfip = pd.read_csv("us_census/zip_fip.csv")
zipfip.head()

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [6]:
# zipfip = zipfip[zipfip["STATE"] == "PA"]
zipfip.head()

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [7]:
unemployment = zipfip.merge(unemployment, left_on='STCOUNTYFP',
             right_on='FIPS_Code', how='left')
unemployment = unemployment[['ZIP', 'Rural_Urban_Continuum_Code_2023', 'Unemployment_rate_2023']]
unemployment = unemployment.rename(columns={
                                   "ZIP": "postal_code", 
                                   "Rural_Urban_Continuum_Code_2023": "rural_urban_continuum_code_2023",
                                   "Unemployment_rate_2023": "unemployment_rate_2023"})
unemployment = unemployment.groupby('postal_code', as_index=False).mean()
unemployment.head()

,postal_code,rural_urban_continuum_code_2023,unemployment_rate_2023
0,501,1.0,3.40
1,601,4.0,7.90
2,602,2.0,7.35
3,603,2.0,7.35
4,604,2.0,7.80


In [8]:
df = demographics.merge(unemployment, left_on='postal_code', right_on='postal_code', how='left')
df.head()

,postal_code,dist_highway,adjusted_gross_income,rural_urban_continuum_code_2023,unemployment_rate_2023
0,1001,1.387035,598807,2.0,4.30
1,1002,14.438177,989558,3.5,2.90
2,1005,16.788339,164207,2.0,3.50
3,1007,13.663839,647074,3.0,2.90
4,1008,2.593655,47826,2.5,3.95


In [9]:
df.to_csv('us_census/demographics_merged.csv', index=False)

In [10]:
df.info()
df.describe().round(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33704 entries, 0 to 33703
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   postal_code                      33704 non-null  int64  
 1   dist_highway                     33704 non-null  float64
 2   adjusted_gross_income            33704 non-null  int64  
 3   rural_urban_continuum_code_2023  33302 non-null  float64
 4   unemployment_rate_2023           33590 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 1.3 MB


,postal_code,dist_highway,adjusted_gross_income,rural_urban_continuum_code_2023,unemployment_rate_2023
count,33704.00,33704.00,33704.00,33302.00,33590.00
mean,49842.61,29.29,425134.10,4.02,3.66
std,27451.11,174.10,756151.94,2.75,1.16
min,1001.00,0.00,577.00,1.00,0.30
25%,27052.75,3.05,32475.00,1.00,2.90
50%,49780.50,11.54,112866.50,3.00,3.50
75%,72210.25,30.05,505229.00,6.33,4.20
max,99929.00,10767.71,15224673.00,9.00,17.30


In [11]:
df["postal_code"].nunique()

33704